In [1]:
#export
from torch import nn
from torch.nn import functional as F
from torch.nn.modules import activation as torch_act

from loop.annotations import MaybeActivation, ListOfModules


__all__ = ['get_activation', 'fc', 'conv2d', 'bottleneck', 'Flatten',
           'AdaptiveConcatPool2d', 'TinyNet']

In [2]:
#export
_public_names = [
    (name, cls) for name, cls in 
    ((name, getattr(torch_act, name)) for name in dir(torch_act) 
     if not name.startswith('_')) 
    if type(cls) == type and issubclass(cls, nn.Module) and cls is not nn.Module]

_name_to_cls = {name.lower(): cls for name, cls in _public_names}

In [3]:
#export
def get_activation(name: str) -> MaybeActivation:
    """Convenience function that creates activation function from string.

    A string can include not only an activation function name but also positional and/or 
    keyword parameters accepted by function initialization. Therefore, the string could 
    have one of the following formats:
        * name
        * name:value1;value2;...
        * name:value1;param2=value2;...
        * name:param1=value1;param2=value2;...

    If not a string was provided but instance of torch.nn.Module, then the object is returned
    as is.

    Examples:
        get_activation('relu')
        get_activation('relu:inplace=True')
        get_activation('prelu:3')

    """
    if isinstance(name, nn.Module):
        return name

    if name is None or name.lower() in ('linear', 'none'):
        return None

    args, kwargs = [], {}
    if ':' in name:
        name, params = name.split(':')
        for param in params.split(';'):
            if '=' in param:
                key, value = param.split('=')
                kwargs[key] = value
            else:
                args.append(param)
    cls = _name_to_cls[name.lower()]
    return cls(*args, **kwargs)

In [4]:
#export
act = get_activation

In [5]:
assert isinstance(act('relu'), torch_act.ReLU)
assert isinstance(act('ReLU'), torch_act.ReLU)
assert isinstance(act('RELU'), torch_act.ReLU)
assert isinstance(act('relu:inplace=True'), torch_act.ReLU)

In [6]:
#export
class TinyNet(nn.Module):
    """Simplistic convolution network.

    Something suitable for tests and simple datasets training but probably nothing more.
    """
    def __init__(self, n_channels=3, n_out=10, activation=None):
        super().__init__()
        self.conv1 = nn.Conv2d(n_channels, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(500, 50)
        self.fc2 = nn.Linear(50, n_out)
        self.activation = activation

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        if self.activation is not None:
            x = self.activation(x)
        return x

In [7]:
#export
class AdaptiveConcatPool2d(nn.Module):
    """Applies average and maximal adaptive pooling to the tensor and
    concatenates results into a single tensor.

    The idea is taken from fastai library.
    """
    def __init__(self, size=1):
        super().__init__()
        self.avg = nn.AdaptiveAvgPool2d(size)
        self.max = nn.AdaptiveMaxPool2d(size)

    def forward(self, x):
        return torch.cat([self.max(x), self.avg(x)], 1)

In [8]:
#export
class Flatten(nn.Module):
    """Converts N-dimensional tensor into 'flat' one."""

    def __init__(self, keep_batch_dim=True):
        super().__init__()
        self.keep_batch_dim = keep_batch_dim

    def forward(self, x):
        if self.keep_batch_dim:
            return x.view(x.size(0), -1)
        return x.view(-1)

In [ ]:
#export
class SeparableConv(nn.Module):
    """Simple implementation of N-d separable convolution."""
    
    def __init__(self, ni, no, kernel=3, stride=1, pad=0, conv=nn.Conv1d):
        super().__init__()
        self.depthwise = conv(ni, ni, kernel, stride, padding=pad, groups=ni)
        self.pointwise = conv(ni, no, kernel_size=1)
    
    def forward(self, x):
        return self.pointwise(self.depthwise(x))

In [9]:
#export
def fc(ni: int, no: int, bias: bool=True, bn: bool=True, activ: str='linear',
       dropout: float=None) -> ListOfModules:
    """Convenience function that creates a linear layer instance with the 'batteries included'.

    The list of created layers:
        * Linear
        * (optionally) BatchNorm
        * (optionally) Dropout
        * (optionally) Activation function

    """
    layers = [nn.Linear(ni, no, bias)]
    if bn:
        layers.append(nn.BatchNorm1d(no))
    if activ is not None:
        func = act(activ)
        if func is not None:
            layers.append(func)
    if dropout is not None:
        layers.append(nn.Dropout(dropout))
    return layers

In [10]:
#export
def conv2d(ni: int, no: int, kernel: int, stride: int, groups: int=1, 
           lrn: bool=False, bn: bool=False, pad: int=0, pool: tuple=None,
           activ: str='prelu') -> ListOfModules:
    """Convenience function that creates a 2D conv layer with the 'batteries included'.

    The list of created layers:
        * Convolution layer
        * (optionally) Activation
        * (optionally) Local response norm OR Batch norm
        * (optionally) Pooling

    """
    bias = not (lrn or bn)
    layers = [nn.Conv2d(ni, no, kernel, stride, pad, bias=bias, groups=groups)]
    func = act(activ)
    if func is not None:
        layers += [func]
    elif lrn:
        layers.append(nn.LocalResponseNorm(2))
    if pool is not None:
        layers.append(nn.MaxPool2d(*pool))
    return layers

In [ ]:
#export
def sepconv(ni: int, no: int, kernel: int, stride: int, 
            pad: int, drop: float=None, activ: str='relu',
            conv: nn.Module=nn.Conv1d) -> ListOfModules:
    """Convenience function that creates conv layer with the 'batteries included'."""
    
    assert drop is None or (0.0 < drop < 1.0)
    layers = [_SeparableConv(ni, no, kernel, stride, pad, conv=conv)]
    layers.append(act(activ))
    if drop is not None:
        layers.append(nn.Dropout(drop))
    return layers

In [11]:
#export
def bottleneck() -> ListOfModules:
    """A 'bridge' from convolutional blocks to fully-connected layers."""
    return [AdaptiveConcatPool2d(1), Flatten()]